<a href="https://colab.research.google.com/github/JackieLee23/HPML-Final-Project/blob/main/optimizer_comparison_figs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#Wandb constants
entity="HPML-final-team"
project="HPML Final Project"

In [2]:
%pip install wandb

In [3]:
import wandb
wandb.login()
api = wandb.Api()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jcl2259 (HPML-final-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
# Initialize wandb run to log
wandb.init(name = "optimizer_comps", project = project, entity = entity)

# Training curves

In [4]:
run_names_dict = {
    "Adam": ["optimizer_Adam", "optimizer_Adam_2", "optimizer_Adam_3"],
    "AdamW": ["optimizer_AdamW", "optimizer_AdamW_2", "optimizer_AdamW_3"],
    "SGD": ["optimizer_SGD", "optimizer_SGD_2", "optimizer_SGD_3"],
}
optimizer_train_dict = {}

In [5]:
for optimizer_name, run_names in run_names_dict.items():
    train_losses = []
    for run_name in run_names:
        run = api.runs(f"{entity}/{project}",
                        filters = {"display_name": run_name})[0]
        history = run.history(keys = ["train_loss"])
        train_losses.append(history["train_loss"].values)

    train_losses = np.array(train_losses)
    optimizer_train_dict[optimizer_name] = train_losses.mean(axis = 0)

In [11]:
epochs = np.arange(1, len(optimizer_train_dict['Adam']) + 1)
fig = plt.figure(figsize = (10, 7))
plt.plot(epochs, optimizer_train_dict['Adam'], '-', label = 'Adam', color = "black")
plt.plot(epochs, optimizer_train_dict['AdamW'], '--', label = 'AdamW', color = "red")
plt.plot(epochs, optimizer_train_dict['SGD'], '-.', label = 'SGD', color = "blue")

plt.xlabel("Epoch", fontsize = 20)
plt.ylabel("Train Loss", fontsize = 20)
plt.yscale("log")
plt.grid(True)
plt.legend()

wandb.log({"Train_loss": wandb.Image(fig)})
plt.close(fig)

# Validation curves

In [7]:
optimizer_val_dict = {}

for optimizer_name, run_names in run_names_dict.items():
    val_losses = []
    for run_name in run_names:
        run = api.runs(f"{entity}/{project}",
                        filters = {"display_name": run_name})[0]
        history = run.history(keys = ["val_loss"])
        val_losses.append(history["val_loss"].values)

    val_losses = np.array(val_losses)
    optimizer_val_dict[optimizer_name] = val_losses.mean(axis = 0)

In [12]:
epochs = np.arange(1, len(optimizer_train_dict['Adam']) + 1)
fig = plt.figure(figsize = (10, 7))
plt.plot(epochs, optimizer_val_dict['Adam'], '-', label = 'Adam', color = "black")
plt.plot(epochs, optimizer_val_dict['AdamW'], '--', label = 'AdamW', color = "red")
plt.plot(epochs, optimizer_val_dict['SGD'], '-.', label = 'SGD', color = "blue")

plt.xlabel("Epoch", fontsize = 20)
plt.ylabel("Validation Loss", fontsize = 20)
plt.yscale("log")
plt.grid(True)
plt.legend()
wandb.log({"Validation_loss": wandb.Image(fig)})
plt.close(fig)